# Depth-first & Breadth-first Search in Python

Source [here](https://eddmann.com/posts/depth-first-search-and-breadth-first-search-in-python/)

---

Note: not ideal implementation as it yields sets, instead of lists (preserving order of discovery). Also, in the original article, the author names a variable after the built-in function `next`, bad practice!

In [2]:
graph = {'A': {'B', 'C'},
         'B': {'A', 'D', 'E'},
         'C': {'A', 'F'},
         'D': {'B'},
         'E': {'B', 'F'},
         'F': {'C', 'E'}}

In [58]:
def dfs(graph, start):
    visited, stack = set(), [start]
    print('starting with:', start)
    while stack:
        vertex = stack.pop()
        print('now checking vertex {} \nstack: {:<10} \nvisited: {}'.format(vertex, repr(stack), visited))
        if vertex not in visited:
            print(vertex, 'not visited, saving & stacking (neighbours - visited):', graph[vertex])
            visited.add(vertex)
            stack.extend(graph[vertex] - visited)
            print('stack now:', stack)
        else:
            print('already visited vertex', vertex, '> skipping!')
        print()
    return visited

![graph](graph.png)

In [59]:
print(dfs(graph, 'A'))

starting with: A
now checking vertex A 
stack: []         
visited: set()
A not visited, saving & stacking (neighbours - visited): {'C', 'B'}
stack now: ['C', 'B']

now checking vertex B 
stack: ['C']      
visited: {'A'}
B not visited, saving & stacking (neighbours - visited): {'A', 'D', 'E'}
stack now: ['C', 'D', 'E']

now checking vertex E 
stack: ['C', 'D'] 
visited: {'A', 'B'}
E not visited, saving & stacking (neighbours - visited): {'F', 'B'}
stack now: ['C', 'D', 'F']

now checking vertex F 
stack: ['C', 'D'] 
visited: {'A', 'E', 'B'}
F not visited, saving & stacking (neighbours - visited): {'C', 'E'}
stack now: ['C', 'D', 'C']

now checking vertex C 
stack: ['C', 'D'] 
visited: {'A', 'E', 'B', 'F'}
C not visited, saving & stacking (neighbours - visited): {'A', 'F'}
stack now: ['C', 'D']

now checking vertex D 
stack: ['C']      
visited: {'F', 'C', 'B', 'A', 'E'}
D not visited, saving & stacking (neighbours - visited): {'B'}
stack now: ['C']

now checking vertex C 
stack: []   

In [70]:
# recusrive version
def dfs(graph, start, visited=None):
    if visited is None:
        visited = set()
    print('start', start)
    print('visited:', visited)
    visited.add(start)
    print('adjacent nodes:', graph[start])
    print('next (adjacent - visited):', graph[start] - visited)
    print()
    for node in graph[start] - visited:
        dfs(graph, node, visited)
    return visited

![graph](graph.png)

In [71]:
dfs(graph, 'A')

start A
visited: set()
adjacent nodes: {'C', 'B'}
next (adjacent - visited): {'C', 'B'}

start C
visited: {'A'}
adjacent nodes: {'A', 'F'}
next (adjacent - visited): {'F'}

start F
visited: {'A', 'C'}
adjacent nodes: {'C', 'E'}
next (adjacent - visited): {'E'}

start E
visited: {'A', 'C', 'F'}
adjacent nodes: {'F', 'B'}
next (adjacent - visited): {'B'}

start B
visited: {'A', 'C', 'E', 'F'}
adjacent nodes: {'A', 'D', 'E'}
next (adjacent - visited): {'D'}

start D
visited: {'F', 'C', 'B', 'A', 'E'}
adjacent nodes: {'B'}
next (adjacent - visited): set()

start B
visited: {'F', 'D', 'C', 'B', 'A', 'E'}
adjacent nodes: {'A', 'D', 'E'}
next (adjacent - visited): set()



{'A', 'B', 'C', 'D', 'E', 'F'}

---

# Paths

In [97]:
def dfs_paths(graph, start, goal):
    stack = [(start, [start])]
    print('start:', start)
    while stack:
        (vertex, path) = stack.pop()
        print()        
        print('at vertex: {} | current path: {}'.format(vertex, repr(path)))
        for node in graph[vertex] - set(path):
            if node == goal:
                print('arrived!, node: {} | path: {}'.format(node, repr(path+[node])))
                print('-------------------------')
                yield path + [node]
            else:
                print('appending node {} to path'.format(node))
                stack.append((node, path + [node]))

![graph](graph.png)

In [98]:
list((dfs_paths(graph, 'A', 'F')))

start: A

at vertex: A | current path: ['A']
appending node C to path
appending node B to path

at vertex: B | current path: ['A', 'B']
appending node D to path
appending node E to path

at vertex: E | current path: ['A', 'B', 'E']
arrived!, node: F | path: ['A', 'B', 'E', 'F']
-------------------------

at vertex: D | current path: ['A', 'B', 'D']

at vertex: C | current path: ['A', 'C']
arrived!, node: F | path: ['A', 'C', 'F']
-------------------------


[['A', 'B', 'E', 'F'], ['A', 'C', 'F']]

In [120]:
# recursive implementation

def dfs_paths(graph, start, goal, path=None):
    if path is None:
        path = [start]
    if start == goal:
        print('done!, path:', path)
        print('---------------')
        print()
        yield path
    print('currently at: {} | goal: {}'.format(start, goal))
    print('path:', path)
    print('adjacent:', graph[start])
    print('next (adjacent - path):', graph[start] - set(path))
    print()
    for node in graph[start] - set(path):
        yield from dfs_paths(graph, node, goal, path + [node])

![graph](graph.png)

In [121]:
print(list(dfs_paths(graph, 'C', 'F')))

currently at: C | goal: F
path: ['C']
adjacent: {'A', 'F'}
next (adjacent - path): {'A', 'F'}

currently at: A | goal: F
path: ['C', 'A']
adjacent: {'C', 'B'}
next (adjacent - path): {'B'}

currently at: B | goal: F
path: ['C', 'A', 'B']
adjacent: {'A', 'D', 'E'}
next (adjacent - path): {'D', 'E'}

currently at: D | goal: F
path: ['C', 'A', 'B', 'D']
adjacent: {'B'}
next (adjacent - path): set()

currently at: E | goal: F
path: ['C', 'A', 'B', 'E']
adjacent: {'F', 'B'}
next (adjacent - path): {'F'}

done!, path: ['C', 'A', 'B', 'E', 'F']
---------------

currently at: F | goal: F
path: ['C', 'A', 'B', 'E', 'F']
adjacent: {'C', 'E'}
next (adjacent - path): set()

done!, path: ['C', 'F']
---------------

currently at: F | goal: F
path: ['C', 'F']
adjacent: {'C', 'E'}
next (adjacent - path): {'E'}

currently at: E | goal: F
path: ['C', 'F', 'E']
adjacent: {'F', 'B'}
next (adjacent - path): {'B'}

currently at: B | goal: F
path: ['C', 'F', 'E', 'B']
adjacent: {'A', 'D', 'E'}
next (adjacent

---

# Breadth-first Search

In [130]:
def bfs(graph, start):
    visited, queue = set(), [start]
    print('start:', start)
    print('visited: {} | queue: {}'.format(visited, queue))
    print()
    while queue:
        vertex = queue.pop(0)
        print('next: {} | queue: {}'.format(vertex, queue)) 
        print('visited', visited)
        if vertex not in visited:
            visited.add(vertex)
            print('now adding adjacent {} to queue'.format(graph[vertex]-visited))
            queue.extend(graph[vertex] - visited)
        print()
    return visited

![graph](graph.png)

In [131]:
bfs(graph, 'A')

start: A
visited: set() | queue: ['A']

next: A | queue: []
visited set()
now adding adjacent {'C', 'B'} to queue

next: C | queue: ['B']
visited {'A'}
now adding adjacent {'F'} to queue

next: B | queue: ['F']
visited {'A', 'C'}
now adding adjacent {'D', 'E'} to queue

next: F | queue: ['D', 'E']
visited {'A', 'C', 'B'}
now adding adjacent {'E'} to queue

next: D | queue: ['E', 'E']
visited {'A', 'C', 'B', 'F'}
now adding adjacent set() to queue

next: E | queue: ['E']
visited {'F', 'D', 'C', 'B', 'A'}
now adding adjacent set() to queue

next: E | queue: []
visited {'F', 'D', 'C', 'B', 'A', 'E'}



{'A', 'B', 'C', 'D', 'E', 'F'}

In [151]:
def bfs_paths(graph, start, goal):
    queue = [(start, [start])]
    print('start: {} | goal: {} | queue: {}'. format(start, goal, queue))
    while queue:
        (vertex, path) = queue.pop(0)
        print()
        print('at vertex:', vertex)
        for node in graph[vertex] - set(path):
            if node == goal:
                print()
                print('done!, path:', path+[node])
                print('-'*40)
                yield path + [node]
            else:
                print()
                print('queue:', queue)
                print('appending node {} & path {} to queue.'.format(node, path+[node]))
                queue.append((node, path + [node]))

![graph](graph.png)

In [152]:
print(list(bfs_paths(graph, 'A', 'F')))

start: A | goal: F | queue: [('A', ['A'])]

at vertex: A

queue: []
appending node C & path ['A', 'C'] to queue.

queue: [('C', ['A', 'C'])]
appending node B & path ['A', 'B'] to queue.

at vertex: C

done!, path: ['A', 'C', 'F']
----------------------------------------

at vertex: B

queue: []
appending node D & path ['A', 'B', 'D'] to queue.

queue: [('D', ['A', 'B', 'D'])]
appending node E & path ['A', 'B', 'E'] to queue.

at vertex: D

at vertex: E

done!, path: ['A', 'B', 'E', 'F']
----------------------------------------
[['A', 'C', 'F'], ['A', 'B', 'E', 'F']]
